# wha fakdsfj  alsfjasl
> "Awesonononon sdfjls"

toc: true
badges: true
comments: true
image: https://i.imgur.com/6zM7JBq.png


# Scraping Data Science Jobs on in.indeed.com

## Introduction 

Indeed.com is a worldwide employment website for job listings. Globally, it hosts millions of job listings on thousands of jobs. In this project, we are interested in the 'Data Science' related job listings on https://in.indeed.com/ . Thus, we'll scrape the website for this information and save that to a csv file for future use. In order to do this, we'll use the following tools:
 - Python as the programming language
 - Requests library for downloading the webpage contents
 - BeautifulSoup library for finding and accessing the relevant information from the downloaded webpage.
 - Numpy library for handling missing values.
 - Pandas library for saving the accessed information to a csv file.
     

Here are the steps we'll follow:
 
 
 - We'll scrape first 10 pages from https://in.indeed.com/jobs?q=data%20science
 - We'll get a list of all 15 jobs on each page.
 - For each job, we'll grab the Job Title, Salary, Location, Company Name, and Company Rating.
 - We'll save all of the information in a csv file in the following format:
 
 ```
 Title,Salary,Location,Company Name,Company Rating
 Data science,"₹35,000 - ₹45,000 a month","Mumbai, Maharashtra",V Digitech Sevices
 Data Science ( 2 - 8 Yrs) - Remote,"₹8,00,000 - ₹20,00,000 a year",Remote,ProGrad
 Data Science Internship,"₹10,000 a month","Gurgaon, Haryana",Zigram
 ```
        

## Initial Setup

Import the required libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from numpy import nan

Set up base URL and the user-agent.

In [2]:
# The base_url is grabbed after searching 'Data Science' on in.indeed.com
# The start value in the base_url will increment by 10 to access each following page.

base_url = "https://in.indeed.com/jobs?q=data%20science&start={}"

header = {"User-Agent": "Mozilla/5.0"}

Create a dictionary to save all information.

In [3]:
jobs = {"Job Title": [],
        "Salary": [],
        "Location": [],
        "Company Name": [],
        "Company Rating": []}

## Scrape the search result webpage.

#### Download webpage and create a BeautifulSoup object

In [4]:
def get_soup(url):
    '''
    This function will download the webpage for the url supplied as argument
    and return the BeautifulSoup object for the webpage which can be used to 
    grab required information for the webpage.
    '''
    response = requests.get(url, "html.parser", headers = header)
    
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(url))
        
    soup = BeautifulSoup(response.text)
    
    return soup

Example for `get_soup`

In [5]:
soup = get_soup(base_url.format(10))
type(soup)

bs4.BeautifulSoup

#### Create a transform function
Now, we'll create a transform function to grab the list of jobs from the result webpage.

To do this, we'll pick `td` tags with the class: `resultContent`

![](https://i.imgur.com/8MXbGpC.png)

In [6]:
def transform(soup):
    # find all the job listings on the webpage.
    jobs_tags = soup.find_all("td", class_ = "resultContent")
    
    # for each job, call helper functions to grab information about the job
    # and save that to jobs dictionary.
    for job in jobs_tags:
        jobs["Job Title"].append(get_job_title(job))
        jobs["Salary"].append(get_job_salary(job))
        jobs["Location"].append(get_company_location(job))
        jobs["Company Name"].append(get_company_name(job))
        jobs["Company Rating"].append(get_company_rating(job))

Example for finding the job tags

In [7]:
job_tags = soup.find_all("td", class_ = "resultContent")
print(len(job_tags))


15


In [8]:
print(job_tags[0].text)

newData Scientist IIMasterCard4.1Gurgaon, Haryana


### Create helper functions
Create helper functions to grab job information for each job and store that in the jobs dictionary.

First grab Job Title.

In [9]:
def get_job_title(job):
    '''
    Function to grab the job title.
    Because some job titles have a prefix new in their job titles,
    this function will automatically detect this prefix and return
    the title sans 'new' in the job title.
    '''
    title = job.find(class_ = "jobTitle").text
    if title[:3] == "new":
        return title[3:]
    else:
        return title
    
get_job_title(job_tags[0])


'Data Scientist II'

Now, we'll grab the job salary, if the listing has one.

In [10]:
def get_job_salary(job):
    
    salary = job.find("div", class_ = "salary-snippet")
    if salary:
        return salary.text
    else:
        return nan
    
get_job_salary(job_tags[1])


nan

Similarly, we'll grab the company name, location, and its rating.

In [11]:
def get_company_name(job):
    '''
    Returns the company name for the supp'''
    return job.find(class_ = "companyName").text

    
def get_company_location(job):
    '''
    Returns the company location for the supplied job tag
    '''
    return job.find(class_ = "companyLocation").text


def get_company_rating(job):
    '''
    Returns the company rating for the supplied job tag
    '''
    rating = job.find(class_ = "ratingNumber")
    if rating:
        return float(rating.text)
    else:
        return nan


print(get_company_name(job_tags[0]), 
      get_company_location(job_tags[0]), 
      get_company_rating(job_tags[0]),
     sep = "\n")

MasterCard
Gurgaon, Haryana
4.1


### Putting it all together

We'll use a `for loop` to loop through 10 search result pages. Within this loop, we can apply the `get_soup` function to download these pages and the `transform` function to parse through all job listings from these pages and save the information in the `jobs` dictionary.
We'll then use this dictionary to create a pandas DataFrame, which can then be saved to a csv file.

In [12]:
for page in range(0, 110, 10):
    print(f"Scraping page {page}...")
    soup = get_soup(base_url.format(page))
    transform(soup)

Scraping page 0...
Scraping page 10...
Scraping page 20...
Scraping page 30...
Scraping page 40...
Scraping page 50...
Scraping page 60...
Scraping page 70...
Scraping page 80...
Scraping page 90...
Scraping page 100...


In [13]:
jobs_df = pd.DataFrame(jobs)

In [14]:
jobs_df.head()

,Job Title,Salary,Location,Company Name,Company Rating
0,Analyst-Data Science,NaN,"Bengaluru, Karnataka+1 location",Amex,4.1
1,Data Science,"₹12,00,000 - ₹18,00,000 a year","Bengaluru, Karnataka",Onward Group,NaN
2,Data Scientist - ML,NaN,"Bengaluru, Karnataka","JPMorgan Chase Bank, N.A.",3.9
3,Data Scientist,NaN,"Pune, Maharashtra+1 location",Maersk,4.1
4,Data Engineer,NaN,"Gurgaon, Haryana+3 locations",NatWest Group,3.3


In [16]:
jobs_df.to_csv("Data_Science_jobs_from_indeed.com.csv", index = None, encoding = "utf-8")